## Опис проблем з візуалізацією

![example](img/1.png)

* Підписи міст розташовані вертикально, через це важко їх прочитати

* Незважаючи на те, що кольори легко відрізнити один від одного, вони не дуже підходять, через велику кількість стовпців у очах рябить, також це перешкоджає можливості порівняння значень.

* Незважаючи на зрозумілу ситему відступів, через намагання показати всі дані у одному недовгому рядку, стовпці стали дуже тонкими, що негативно впливає на можливість оцінити вклад конкретного сектору і відповідно порівнювати їх.

## Альтернативний варіант візуалізації 

In [1]:
import pandas as pd
import altair as alt
import numpy as np

In [2]:
data = pd.read_excel('../data/1.4_02.xlsx')
data.head()

,"Регіон, рік",сільське господарство,промисловість,будівництво,послуги,державне управління
0,"Україна, 2010",8.3,25.9,3.3,57.3,5.2
1,"Україна, 2017",12.1,25.2,2.5,53.7,6.5
2,"Вінницька, 2010",21.6,19.3,4.1,47.4,7.6
3,"Вінницька, 2017",28.1,21.4,2.0,39.8,8.7
4,"Волинська, 2010",16.9,15.6,3.2,57.7,6.6


In [3]:
data[['region', 'year']] = data['Регіон, рік'].str.split(',', expand=True)
data.drop(['Регіон, рік'], axis = 1, inplace = True)
data['year'] = data['year'].astype(int)
data.head()

,сільське господарство,промисловість,будівництво,послуги,державне управління,region,year
0,8.3,25.9,3.3,57.3,5.2,Україна,2010
1,12.1,25.2,2.5,53.7,6.5,Україна,2017
2,21.6,19.3,4.1,47.4,7.6,Вінницька,2010
3,28.1,21.4,2.0,39.8,8.7,Вінницька,2017
4,16.9,15.6,3.2,57.7,6.6,Волинська,2010


In [4]:
data = data.melt(id_vars = ['region', 'year'], var_name = 'Parts', value_name = 'Value' )
data['Value'] = data['Value'].astype(float)
data['Value'] = data['Value'].replace(0, np.nan)
data.head()

,region,year,Parts,Value
0,Україна,2010,сільське господарство,8.3
1,Україна,2017,сільське господарство,12.1
2,Вінницька,2010,сільське господарство,21.6
3,Вінницька,2017,сільське господарство,28.1
4,Волинська,2010,сільське господарство,16.9


In [5]:
bars = alt.Chart(data).mark_bar().encode(
    x = alt.X('Value:Q', scale = alt.Scale(nice = False)),
    y = alt.Y('region:O', sort = alt.Sort(field = 'Value', op = 'max', order = 'descending')),
    color = alt.Color('Parts:N', legend=alt.Legend(title='', orient = 'bottom')),
    tooltip = alt.Tooltip('Value'),
    order = alt.Order(field = 'Value', type = 'quantitative', sort = 'descending')
)

text = alt.Chart(data).mark_text(dx=-15, dy=3, fontSize = 13, fontWeight=600, color='white').encode(
    x=alt.X('Value:Q', stack='zero'),
    y=alt.Y('region:O'),
    detail=alt.Detail('Parts:N'),
    text=alt.Text('Value:Q', format='.1f'),
    order = alt.Order(field = 'Value', type = 'quantitative', sort = 'descending')
).transform_filter(
    alt.FieldRangePredicate(field='Value', range=[5.9, 86.3]))

chart = (bars + text).properties(width = 800/2, height = 700).facet(facet = alt.Facet('year:N', title = 'Рік'))

chart = chart.properties(
    title = alt.TitleParams(
        text = 'Зміна структури ВДВ у розрізі регіонів',
        subtitle = 'Порівняння 2010 та 2017 років'
    ),
    padding = 20
).configure_view(
    strokeWidth = 0
).configure_axis(
    domain = False,
    ticks = False,
    title = None,
    labelPadding = 10
).configure_title(
    anchor = 'start',
    frame = 'group',
    fontSize = 24,
    subtitleFontSize = 16
)

In [6]:
chart.display(actions = False, renderer = 'png')

alt.FacetChart(...)

* Підписи міст розташовані на осі Y вертикально, їх легко прочитати та асоціювати з відповідними стовпцями справа.

* Я вирішила використати кольори за замовчуванням, на мою думку, з ними візуалізація виглядає краще.

* Також посортувала кольорові прямокутники від найбільшого до найменшого, таким чином краще видно різницю у сруктурі між регіонами та роками.

* Вирішила додати підписи для деяких прямокутників для того, щоб краще можна було орієнтуватися та співвідносити довжину прямокутників з початковими цифровими даними.

* Для того, щоб дізнатися значення прямокутників, на яких не відображаються числа, потрібно просто навести на них і ця інформація з'явиться на екрані. Загалом ця функція працює для всіх прямокутників.

* На мою думку, таке росташування стовпців дозволило легше порівнювати структуру як між регіонами (вертикальне розташування стовпців), так і між роками (горизонтальне розташування).